In [17]:
#Simple Voice Assistant
#Wojciech Biskup


! pip install SpeechRecognition
! pip install sounddevice 
! pip install pocketsphinx
! pip install speake3
! pip install bingpython


# install also sound/recording libraries
# sudo apt-get install libportaudio2
# sudo apt-get install libasound-dev
# sudo apt-get install espeak
# for macOS with ARM chips use appkit library

In [18]:
#Choose operating system:
#For windows change "OS_TYPE" to 0 if MacOS switch to 1
OS_TYPE = 1

In [36]:
#all needed library
import speech_recognition as sr
import sounddevice as sd
import soundfile as sf
import numpy as np
import scipy.io.wavfile as wav
import speake3
import re
import time
if OS_TYPE == 1:
    from AppKit import NSSpeechSynthesizer

In [20]:
#mathhh
def convert_string(expression):
    pattern = r'(\d+)\s+(plus|minus|times|divided by)\s+(\d+)'
    converted_expression = re.sub(pattern, r'\1 \2 \3', expression)

    # Replace words with corresponding symbols
    converted_expression = converted_expression.replace('plus', '+')
    converted_expression = converted_expression.replace('minus', '-')
    converted_expression = converted_expression.replace('times', '*')
    converted_expression = converted_expression.replace('divided by', '/')
    return converted_expression

def calculate_expression(expression):
    try:
        result = eval(expression)
        return result
    except:
        return "Error: Invalid expression"

In [21]:
#Possible actions for voice assistant
if OS_TYPE == 1:
    def readText(phrase):
        speechSynthesizer = NSSpeechSynthesizer.alloc().init()
        speechSynthesizer.startSpeakingString_(phrase)
elif OS_TYPE == 0:
    def readText(phrase):
        engine = speake3.Speake()
        engine.set('voice', 'en')
        engine.set('speed', '104')
        engine.set('pitch', '99')
        engine.say(phrase)
        engine.talkback()
    
def welcome():
    welcome = "Good morning Human!"
    readText(welcome)
    
def goodbye():
    goodbye = "Have a nice day!"
    readText(goodbye)
    
def calculate(phrase):
    string = convert_string(phrase)
    calculate = calculate_expression(string)
    response = "The " + str(string) + " equals to " + str(calculate)
    print(response)
    readText(response)
    
def tags():
    #code...
    print('Hi')
    
def askgpt(question):
    response = openai.Completion.create(
        engine='davinci',
        prompt=question,
        max_tokens=100,
        temperature=0.7,
        n=1,
        stop=None,
        timeout=10,
    )
    answer = response.choices[0].text.strip()
    readText(answer)

In [65]:
#User input
def UserInput(time):
    fs=44100
    duration = time  # seconds
    myrecording = sd.rec(duration * fs, samplerate=fs, channels=1,dtype='float64')
    print("Recording Audio, please tell me your choice")
    sd.wait()
    print("Audio recording complete")
    sd.wait()
    filename = 'recording.wav'
    sf.write(filename, myrecording, fs)
    r = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio = r.record(source)
    recognizedText =  r.recognize_google(audio)
    recognizedTextAsString = str(recognizedText)
    print("User: "+recognizedTextAsString)
    return recognizedTextAsString

In [66]:
#Initialize simple voice assistant
flag = True
#Welcome
welcome = "Hello, I'm an simple voice assistant. You can ask me following things: 1 - calculate something 2 - read tags with trained model or 3 - Ask question online. Please tell me number of action that you are intrested in, if you want me to repeat possible actions say 4"
welcome2 = "You can ask me following things: 1 - calculate something 2 - read tags with trained model or 3 - Ask question online. Please tell me number of action that you are intrested in, if you want me to repeat possible actions say 4"
readText(welcome)
time.sleep(18)
#User choice
user_choice = UserInput(5)
while flag is True:
    if user_choice == "4":
        readText(welcome2)
        time.sleep(15)
        user_choice = UserInput()
        flag = True
    else:
        flag = False
if user_choice == "1":
    calculate_info = "Please tell me equation to calculate, you can use following operations: addition, subtraction, multiplication and division"
    readText(calculate_info)
    time.sleep(9)
    equation = UserInput(10)
    calculate(equation)

Recording Audio, please tell me your choice
Audio recording complete
User: 1
Recording Audio, please tell me your choice
Audio recording complete
User: 5 + 5 * 6
The 5 + 5 * 6 equals to 35
